# Einstein constraints on a Boyer-Lindquist slice of Kerr spacetime

This notebook demonstrates a few capabilities of SageMath in computations regarding hypersurfaces of a pseudo-Riemannian manifold. The corresponding tools have been developed in the framework of the  [SageManifolds](https://sagemanifolds.obspm.fr) project.

In [ ]:
version()

First we set up the notebook to display maths with LaTeX rendering:

In [ ]:
%display latex

We ask for running computations in parallel on 8 threads:

In [ ]:
Parallelism().set(nproc=8)

## Kerr spacetime $(M,g)$

The mass parameter $m$ and spin parameter $a$ of Kerr solution:

In [ ]:
m, a = var('m a', domain='real')
assume(a<m)

The Kerr spacetime declared as a 4-dimensional Lorentzian manifold:

In [ ]:
M = Manifold(4, 'M', structure='Lorentzian')
print(M)

On $M$, we consider the *"rational-polynomial" coordinates* $(t,r,y,\phi)$ inherited from the standard **Boyer-Lindquist coordinates** $(t,r,\theta,\phi)$ via $y=\cos\theta$:

In [ ]:
BL.<t,r,y,ph> = M.chart(r't r:(0,+oo) y:(-1,1) ph:(0,2*pi):\phi') 
BL

We focus on the exterior of the Kerr black hole, on which $\Delta := r^2 - 2 mr + a^2 > 0$: 

In [ ]:
assume(a^2 -2*m*r+ r^2 > 0)

The Kerr metric:

In [ ]:
g = M.metric()
rho2 = r^2 + (a*y)^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*(1 - y^2)/rho2
g[1,1], g[2,2] = rho2/Delta, rho2/(1 - y^2)
g[3,3] = (r^2 + a^2 + 2*m*r*a^2*(1 - y^2)/rho2)*(1 - y^2)
g.display()

As a test, let us check that $g$ is a solution of the **vacuum Einstein equation**:

In [ ]:
g.ricci().display()

## Spacelike hypersurface $(\Sigma,\gamma)$

We declare $\Sigma$ as a 3-dimensional (immersed) submanifold of $M$ via the keyword `ambient`:

In [ ]:
S = Manifold(3, 'S', latex_name=r'\Sigma', ambient=M, 
             structure="Riemannian", start_index=1)
print(S)

In [ ]:
S

We consider $\Sigma$ to be a constant Boyer-Lindquist coordinate $t$ hypersurface. Natural coordinates on $\Sigma$ are then the Boyer-Lindquist coordinates $(r,y,\phi)$:

In [ ]:
BLS = S.chart(r'r:(0,+oo) y:(-1,1) ph:(0,2*pi):\phi')
BLS

At this stage, the user atlas of $\Sigma$ contains a single chart:

In [ ]:
S.atlas()

The embedding $\Phi: \Sigma \to M$:

In [ ]:
Phi = S.diff_map(M, {(BLS, BL): (t, r, y, ph)}, name='Phi',
                 latex_name=r'\Phi')
Phi.display()

The partial inverse:

In [ ]:
Phi_inv = M.diff_map(S, {(BL, BLS): (r, y, ph)})
Phi_inv.display()

In [ ]:
Phi_inv*Phi == S.identity_map()

$t$ as a scalar field on $M$:

In [ ]:
t_scalar = M.scalar_field(t)
t_scalar.display()

We declare that $\Phi$ is the embedding of $\Sigma$ in $M$ and that $\Sigma$ is a level
surface of $t$:

In [ ]:
S.set_embedding(Phi, inverse=Phi_inv, var=t,
                t_inverse = {t: t_scalar})
print(S)

For some computations, $\Sigma$ is considered as belonging to a foliation of $M$ by
a family of hypersurfaces $(\Sigma_t)_{t\in\mathbb{R}}$. A chart of $M$ adapted to the
foliation is computed by the method `adapted_chart`. In the present case, it is trivial, 
since $t$ is one of the coordinate of the default chart of $M$:


In [ ]:
S.adapted_chart()

In [ ]:
M.atlas()

### The Riemannian metric $\gamma$ induced by $g$ on $\Sigma$

$\gamma$ is the first fundamental form of $\Sigma$:

In [ ]:
gam = S.first_fundamental_form()
gam.display()

It is actually the pullback of $g$ by the embedding $\Phi$:

In [ ]:
gam == Phi.pullback(g)

An alias of `first_fundamental_form` is `induced_metric`:

In [ ]:
gam is S.induced_metric()

The Levi-Civita connection $D$ associated with $\gamma$:

In [ ]:
D = gam.connection(name='D')
print(D)

In [ ]:
D.display()

The unit normal to $\Sigma$ (actually to the foliation $(\Sigma_t)_{t\in\mathbb{R}}$):

In [ ]:
n = S.normal()
print(n)
n.display()

Let us check that $n$ is a unit timelike vector:

In [ ]:
g(n,n).expr()

## The second fundamental form $K$:

In [ ]:
K = S.second_fundamental_form()
print(K)

In [ ]:
K.display()

In [ ]:
K.display_comp()

The metric dual $K^i_{\ \, j} = \gamma^{ik} K_{kj}$:

In [ ]:
Ku = K.up(gam, 0)  # 0 = first index of K
print(Ku)

In [ ]:
Ku.display()

The trace $K := K^i_{\ \, i}$:

In [ ]:
trK = Ku.trace()
print(trK)

We have $K=0$:

In [ ]:
trK.expr()

i.e. $\Sigma$ is a **maximal hypersurface** of $(M,g)$.

## Hamiltonian constraint

The vacuum Hamiltonian constraint equation is 
$$ R + K^2 -K_{ij} K^{ij} = 0 $$
where $R = \mathrm{tr}_\gamma \mathbf{R} = \gamma^{ij} R_{ij}$ is the Ricci scalar of $\gamma$.
Let us first evaluate $R$:

In [ ]:
R = gam.ricci_scalar()
print(R)
R.display()

In [ ]:
R.expr().factor()

The term $K_{ij} K^{ij}$:

In [ ]:
Kuu = Ku.up(gam, 1)
KK = K['_ij']*Kuu['^ij']
print(KK)
KK.display()

Finally the Hamiltonian constraint:

In [ ]:
Ham = R + trK^2 - KK
print(Ham)
Ham.display()

In [ ]:
Ham.expr()

Hence the Hamiltonian constraint is fulfilled. 

## The momentum constraint

In vacuum, the momentum constraint is
$$ D_j K^j_{\ \, i} - D_i K = 0 $$

Given that $D_k K^j_{\ \, i}$ is the tensor $(DK)^j_{\ \, ik}$, we evaluate the momentum constraint 
as follows:

In [ ]:
mom = D(Ku)['^j_{ij}'] - D(trK)
print(mom)

In [ ]:
mom.display()

Hence the momentum constraint is fulfilled. 

*Remark:* an alternative writting for `D(Ku)['^j_{ij}']` is `D(Ku).trace(0,2)`:

In [ ]:
mom == D(Ku).trace(0,2) - D(trK)